In [25]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import sys
import librosa
import numpy as np
import os
import glob
import json

import torch
from text import text_to_sequence, cmudict
from text.symbols import symbols
import commons
import attentions
import modules
import models
import utils



DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


In [26]:
def plot_waveform(waveform, sr, title="Waveform"):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sr

    figure, axes = plt.subplots(num_channels, 1)
    axes.plot(time_axis, waveform[0], linewidth=1)
    axes.grid(True)
    figure.suptitle(title)
    plt.show(block=False)


def plot_spectrogram(specgram, title=None, ylabel="freq_bin"):
    fig, axs = plt.subplots(1, 1)
    axs.set_title(title or "Spectrogram (db)")
    axs.set_ylabel(ylabel)
    axs.set_xlabel("frame")
    im = axs.imshow(librosa.power_to_db(specgram), origin="lower", aspect="auto")
    fig.colorbar(im, ax=axs)
    plt.show(block=False)

In [27]:
# If you are using your own trained model
model_dir = "./"
hps = utils.get_hparams_from_dir(model_dir)
checkpoint_path = utils.latest_checkpoint_path(model_dir)

# If you are using a provided pretrained model
# hps = utils.get_hparams_from_file("./configs/any_config_file.json")
# checkpoint_path = "/path/to/pretrained_model"

model = models.FlowGenerator(
    len(symbols) + getattr(hps.data, "add_blank", False),
    out_channels=hps.data.n_mel_channels,
    **hps.model).to("cuda")

utils.load_checkpoint(checkpoint_path, model)
model.decoder.store_inverse() # do not calcuate jacobians for fast decoding
_ = model.eval()

cmu_dict = cmudict.CMUDict(hps.data.cmudict_path)

# normalizing & type casting
def normalize_audio(x, max_wav_value=hps.data.max_wav_value):
    return np.clip((x / np.abs(x).max()) * max_wav_value, -32768, 32767).astype("int32")

./G_78.pth
INFO:root:Loaded checkpoint './G_78.pth' (iteration 78)


In [28]:
tst_stn = "This is the biggest problem." 

if getattr(hps.data, "add_blank", True):
    text_norm = text_to_sequence(tst_stn.strip(), ['english_cleaners'], cmu_dict)
    text_norm = commons.intersperse(text_norm, len(symbols))
else: # If not using "add_blank" option during training, adding spaces at the beginning and the end of utterance improves quality
    tst_stn = " " + tst_stn.strip() + " "
    text_norm = text_to_sequence(tst_stn.strip(), ['english_cleaners'], cmu_dict)
print(text_norm)
sequence = np.array(text_norm)[None, :]
print("".join([symbols[c] if c < len(symbols) else "<BNK>" for c in sequence[0]]))
x_tst = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
x_tst_lengths = torch.tensor([x_tst.shape[1]]).cuda()

[147, 90, 147, 108, 147, 130, 147, 10, 147, 108, 147, 145, 147, 10, 147, 90, 147, 72, 147, 10, 147, 87, 147, 108, 147, 104, 147, 72, 147, 130, 147, 132, 147, 10, 147, 52, 147, 54, 147, 51, 147, 38, 147, 48, 147, 41, 147, 49, 147, 7, 147]
<BNK>@DH<BNK>@IH1<BNK>@S<BNK> <BNK>@IH1<BNK>@Z<BNK> <BNK>@DH<BNK>@AH0<BNK> <BNK>@B<BNK>@IH1<BNK>@G<BNK>@AH0<BNK>@S<BNK>@T<BNK> <BNK>p<BNK>r<BNK>o<BNK>b<BNK>l<BNK>e<BNK>m<BNK>.<BNK>


In [58]:
with torch.no_grad():
    noise_scale = .667
    length_scale = 1.0
    (y_gen, *_), *_, (attn_gen, *_) = model(x_tst, x_tst_lengths, gen=True)
    import audio_processing as ap
    from stft import STFT
    y_gen=ap.dynamic_range_decompression(y_gen)
    mel=y_gen.detach().cpu()
    mel=mel.numpy()
    mel_basis=librosa.filters.mel(sr=hps.data.sampling_rate, n_fft=hps.data.filter_length, n_mels=hps.data.n_mel_channels)
    covered_mel=librosa.util.nnls(mel_basis, mel)
    audio=librosa.griffinlim(covered_mel,n_iter=60)
    #stft_fn=STFT(hps.data.filter_length, hps.data.hop_length, hps.data.win_length)
    #audio=ap.griffin_lim(torch.tensor(covered_mel),stft_fn,n_iters=60)

#fig, ax = plt.subplots(nrows=2, sharex=True, sharey=True)
#import librosa.display as dp
#dp.waveplot(audio, sr=hps.data.sampling_rate, color='b', ax=ax[0])
#ax[0].set(title='Original', xlabel=None)
#ax[0].label_outer()
ipd.Audio(audio, rate=hps.data.sampling_rate)

/home/caijb/Desktop/glow-tts/models.py:322: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  y_lengths = (y_lengths // self.n_sqz) * self.n_sqz
